# Paraphrase Italian Dataset

In [1]:
import pandas as pd

In [2]:
# Use 'skip' to skip problematic lines and continue parsing
df = pd.read_csv('PACCSS-IT.txt', sep='\t', usecols=range(7), quoting=3)
df.head()

,Sentence_1,Sentence_2,Cosine_Similarity,Confidence,Readability_1,Readability_2,(Readability_1-Readability_2)
0,Ma questo a cosa servirebbe ?,A che servono queste cose ?,0.833333,0.803743,0.994767,0.013161,0.981605
1,"Salve, avrei bisogno di una informazione piutt...",Ho bisogno di una informazione urgente .,0.836660,0.811129,0.982944,0.012627,0.970317
2,Ciao a tutti avrei bisogno di un consiglio .,Ho bisogno di un suo consiglio .,0.755929,0.811662,0.984332,0.014420,0.969912
3,Possibilmente uno che avesse bisogno dell' aiu...,Ho bisogno di un vostro aiuto .,0.801784,0.826830,0.982555,0.014647,0.967908
4,Questa sarebbe una cosa positiva.,Questa era una nuova cosa .,0.771517,0.800786,0.997256,0.030448,0.966808


In [3]:
min_confidence = df['Confidence'].min()
max_confidence = df['Confidence'].max()
min_cosine = df['Cosine_Similarity'].min()
max_cosine = df['Cosine_Similarity'].max()

# Display the results
print(f"Confidence range: {min_confidence} to {max_confidence}")
print(f"Cosine Similarity range: {min_cosine} to {max_cosine}")

Confidence range: 0.800002575466 to 0.9999999
Cosine Similarity range: 0.654653670708 to 0.949288905069


## Positive Class Creation

In [4]:
# drop the last three columns
df = df.iloc[:, :-3]
df.head()

,Sentence_1,Sentence_2,Cosine_Similarity,Confidence
0,Ma questo a cosa servirebbe ?,A che servono queste cose ?,0.833333,0.803743
1,"Salve, avrei bisogno di una informazione piutt...",Ho bisogno di una informazione urgente .,0.836660,0.811129
2,Ciao a tutti avrei bisogno di un consiglio .,Ho bisogno di un suo consiglio .,0.755929,0.811662
3,Possibilmente uno che avesse bisogno dell' aiu...,Ho bisogno di un vostro aiuto .,0.801784,0.826830
4,Questa sarebbe una cosa positiva.,Questa era una nuova cosa .,0.771517,0.800786


In [5]:
confidence_treshold = 0.90
cosine_threshold = 0.80

In [6]:
# Filter the DataFrame based on the thresholds
filtered_df = df[(df['Confidence'] >= confidence_treshold) & (df['Cosine_Similarity'] >= cosine_threshold)]

len(filtered_df)

14545

In [7]:
# pick 100 random samples
sample = filtered_df.sample(n=100, random_state=1)

#show the sample
sample

,Sentence_1,Sentence_2,Cosine_Similarity,Confidence
35173,Ma questo è solo il primo passo .,Ed è solo il primo passo .,0.801784,0.971201
42931,"Posto ai voti , l' emendamento 6.1 , viene res...",Posto ai voti è respinto l' emendamento 1.1 .,0.843274,0.900495
24230,"La Commissione prosegue l' esame , rinviato , ...",La Commissione prosegue l' esame rinviato nell...,0.886405,0.927586
42879,Questo è un punto di estrema importanza .,è un punto di grande importanza .,0.801784,0.964339
59391,"E allora , un "" altro mondo "" sarà possibile .","E allora , un altro mondo sarà possibile .",0.948683,0.931301
...,...,...,...,...
45885,"Si tratta , dunque , di un passaggio decisivo .",Si tratta di un passaggio decisivo .,0.881917,0.951611
15271,"Il fine , però , non giustifica i mezzi .",Il fine non giustifica mai i mezzi .,0.801784,0.950393
12011,Le cose però non starebbero così .,Le cose non stavano esattamente così .,0.857143,0.938938
30628,Questa però è opinione mia .,Questa è una mia opinione .,0.833333,0.957264


In [8]:
# add a column label with value 1
filtered_df['label'] = 1

/var/folders/xz/wxdrzvcj7zzcg3w1l542xxg40000gn/T/ipykernel_23404/457701535.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['label'] = 1


## Negative Class Creation

### Supervied Random Sampling

We take a sentence A from the corpus and randomly sample a sentence B to associate with it. We exclude sentences with a cosine similarity greater than a certain threshold to avoid false negatives. Of course, the sentence pair should not be already present in the dataset. 

In [9]:
random_state = 42

def generate_random_sentence_pair():
    # pick a random Sentence_1
    s1 = df.sample(n=1, random_state=random_state)['Sentence_1'].values[0]
    # pick a random Sentence_2
    s2 = df.sample(n=1, random_state=random_state)['Sentence_2'].values[0]
    return s1, s2


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

threshold = 0.70

negative_df = pd.DataFrame(columns=['Sentence_1', 'Sentence_2', 'Cosine_Similarity', 'label'])

while (len(negative_df) <= len(filtered_df)): # ratio 1:1 with the positive class
    s1, s2 = generate_random_sentence_pair()
    # check if the pair is not already in the corpus
    if not ((df['Sentence_1'] == s1) & (df['Sentence_2'] == s2)).any():
        # check if the pair is not already generated
        if not ((negative_df['Sentence_1'] == s1) & (negative_df['Sentence_2'] == s2)).any():
            # calculate the cosine similarity between s1 and s2
            emb_s1 = model.encode(s1)
            emb_s2 = model.encode(s2)
            cosine_similarity = model.similarity(emb_s1, emb_s2)
            # check if the cosine similarity is less than the threshold
            if cosine_similarity < threshold:
                negative_df.append({'Sentence_1': s1, 'Sentence_2': s2, 'Cosine_Similarity': cosine_similarity, 'label': 0}, ignore_index=True)
                
negative_df.head()

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


KeyboardInterrupt: 

In [ ]:
# save the positive and negative samples to a csv file
filtered_df.to_csv('PACCSS-IT-positive.csv', index=False)
negative_df.to_csv('PACCSS-IT-negative.csv', index=False)